# Dense Document Retriever - Implantação

Ranqueia textos de acordo com um outro texto de entrada através do ranqueador DPR do hugginfaces.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
from model_dense_retriever import EnglishDPRRetriever

In [2]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
import json


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.report_contexts = artifacts["report_contexts"]
        self.inner_batch_size = artifacts["inner_batch_size"]
        self.top = artifacts["top"]
        
        
    
    def build_result_dataframe(self, sim_contexts_ids,scores):
        sim_contexts = [self.report_contexts[i] for i in sim_contexts_ids[0]]
        df = pd.DataFrame({'doc_id':sim_contexts_ids[0],'score':scores[0],'context':sim_contexts})
        df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return df

    def predict(self, X, feature_names, meta=None):

        question = X.copy()[0]
        sim_contexts_ids,scores = self.model(question,self.report_contexts,self.inner_batch_size,self.top)
        df = self.build_result_dataframe(sim_contexts_ids,scores)
        df['question'] = [question]*len(scores[0])
        return df.to_numpy() 

Overwriting Model.py


In [3]:
from Model import Model
model = Model()
model.predict(["What is the best weed herbicide?"],None)

array([[684, -2.7461817264556885,
        ' 7. REFERÊNCIAS   BRIDGES, D.C. Benefits of Triazine Herbicides in Corn and Sorghum Production. In.: LEBARON, H.M.; McFARLAND, J.E. BURNSIDE, O.C. (Ed.). Triazine Herbicides  50 Years Revolutionizing Agriculture. San Diego: Elsevier, 2008. p. 163-174. FRANS, R.; TALBERT, R.; MARX, D.; CROWLEY, H. Experimental design and techniques for measuring and analysing plant responses to weed control practices. In.: CAMPER, N.D. (Ed.). Research methods in weed science. Third ed. USA SOUTHERN WEED SCIENCE SOCIETY, 1986. p. 29-46. FLUFENACET AND ISOXAFLUTOLE. Flufenacet is a synthetic compound of the oxyacetamide family... AgChem Access, 2012. Disponível em: < www.agchemaccess.com/Flufenacet_and_isoxaflutole>. GEIER, P.W.; STAHLMAN, P.W.; FRIHAUF, J.C. KIH-485 and S-metolachlor Efficacy Comparisons in Conventional and No-Tillage Corn. Weed Technology, v. 20, n. 3, p. 622-626, 2006. KING, S.R.; GARCIA, J.O. Annual Broadleaf Control with KIH-485 in Glyphosat